In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
train= pd.read_csv('train.csv')
test= pd.read_csv('test.csv')
#ss= pd.read_csv('sample_submission.csv')


In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train=train[['id','text','target']]
train.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
import pandas as pd
import torch.nn.functional as F
import re
import nltk
from nltk.corpus import stopwords
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score

nltk.download('stopwords')
def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  text = re.sub(r'\s+', ' ', text)
  return text
def remove_stopwards(text):
  stopword = set(stopwords.words('english'))
  words = text.split()
  filtered_words = [word for word in words if word.lower() not in stopword]
  return ' '.join(filtered_words)
train['text'] = train['text'].apply(clean_text)

test['text'] = test['text'].apply(clean_text)

train.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abid.abderrazek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,text,target
0,1,our deeds are the reason of this earthquake ma...,1
1,4,forest fire near la ronge sask canada,1
2,5,all residents asked to shelter in place are be...,1
3,6,13000 people receive wildfires evacuation orde...,1
4,7,just got sent this photo from ruby alaska as s...,1


In [ ]:
USE_ALL_TO_TRAIN=False

In [7]:
# Load model directly
import torch
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'

from transformers import AutoModelForSequenceClassification,AutoTokenizer
from torch.utils.data import Dataset
model_checkpoint='garynguyen1174/disaster_tweet_bert'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


/home/abid.abderrazek/.conda/envs/compet/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from datasets import load_dataset, ClassLabel

def tokenize_dataset(dataset):
    return tokenizer(dataset['text'],padding='max_length',max_length=90)

dataset = load_dataset('csv', data_files={'train': 'train.csv'})
dataset = dataset.rename_column('target', 'label')
dataset = dataset.cast_column('label', ClassLabel(num_classes=2, names=['negative', 'positive']))
dataset = dataset.map(tokenize_dataset, batched=True)
if not USE_ALL_TO_TRAIN:
    dataset = dataset['train'].train_test_split(test_size=0.2, stratify_by_column='label', seed=1)

dataset_test = load_dataset('csv', data_files={'test': 'test.csv'})
dataset_test = dataset_test.map(tokenize_dataset, batched=True)

In [ ]:

tokenized_train_dataset = dataset['train'].remove_columns(['id', 'keyword', 'location', 'text'])

if not USE_ALL_TO_TRAIN:
    tokenized_test_dataset = dataset['test'].remove_columns(['id', 'keyword', 'location', 'text'])

In [11]:

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2).to(device)

In [ ]:

from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score,f1_score
from transformers import DataCollatorWithPadding 
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
# Import wandb
import wandb

config={
    "learning_rate": 1e-5,
    "epochs": 2,
    "batch_size":16,
    "Eval_batch_size":64
    }

wandb.init(
    # set the wandb project where this run will be logged
    project="Disaster Tweets 2",
    name=f"{model_checkpoint}{config['epochs']}-{config['learning_rate']}-batch{config['batch_size']}",     # Custom name for this run

    # track hyperparameters and run metadata
    config=config
)

# Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,padding=True)

def compute_loss(logis,targets):
    loss_fn = CrossEntropyLoss(reduction='sum')
    loss=loss_fn(logis,targets)
    return loss


# Compute Metrics
def compute_metrics(logits, labels):
    logits = logits.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    predictions = logits.argmax(axis=-1)
    return {'f1_score': f1_score(labels, predictions)}

def compute_f1(predictions, labels):
    predictions = predictions.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    return {'f1_score': f1_score(labels, predictions)}



train_dataloader=DataLoader(tokenized_train_dataset,batch_size=config['batch_size'],collate_fn=data_collator)
if not USE_ALL_TO_TRAIN:
    test_dataloader=DataLoader(tokenized_test_dataset,batch_size=config['Eval_batch_size'],collate_fn=data_collator)


### TesT Model from training using Trainer 

In [ ]:
#model = torch.load('rob_good.pth')

## Training

In [15]:
from transformers import AdamW
optimizer=AdamW(model.parameters(),config['learning_rate'],weight_decay=0.01)

steps=0
total_steps=config['epochs']*len(train_dataloader)
for _ in range(1,config['epochs']):
    for idx,val in enumerate(train_dataloader):
        if not USE_ALL_TO_TRAIN:
            if steps==0:
                all_preds=[]
                all_labels=[]
                for t_idx,t_val in enumerate(test_dataloader):
                    val0=t_val
                    out= model(input_ids=val0['input_ids'].to(device),attention_mask=val0['attention_mask'].to(device))
                    targets=val0['labels']
                    preds=torch.argmax(out.logits,dim=1)
                    all_preds.append(preds)
                    all_labels.append(targets)
                all_preds = torch.cat(all_preds)
                all_labels = torch.cat(all_labels)
                f1=compute_f1(all_preds,all_labels)
                print(f'eval {steps}    ', f1)
                wandb.log({"f1": f1})

        out= model(input_ids=val['input_ids'].to(device),attention_mask=val['attention_mask'].to(device))
        targets=val['labels']
        loss=compute_loss(out.logits.to(device),targets.to(device))
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        if not USE_ALL_TO_TRAIN:
            if steps%10==0:
                all_preds=[]
                all_labels=[]
                for t_idx,t_val in enumerate(test_dataloader):
                    val0=t_val
                    out= model(input_ids=val0['input_ids'].to(device),attention_mask=val0['attention_mask'].to(device))
                    targets=val0['labels']
                    preds=torch.argmax(out.logits,dim=1)
                    all_preds.append(preds)
                    all_labels.append(targets)
                all_preds = torch.cat(all_preds)
                all_labels = torch.cat(all_labels)
                f1=compute_f1(all_preds,all_labels)
                print(f'eval {steps} / {total_steps}  ', f1)
                wandb.log({"f1": f1})

        wandb.log({"loss": loss})
        steps+=1

    

/home/abid.abderrazek/.conda/envs/compet/lib/python3.13/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
wandb.finish()
torch.save(model,'best.pth')

loss,▂▄▃▁▂▅▄▂▃▆▃▃▅▂▃▁▃▄█▁▄▃▄▃▅▄▄▂▅▁▅▁▇▂▃▄▁▁▃▄
loss,0.25977


## Evaluation

In [1]:
import torch
model=torch.load('best.pth')

/tmp/slurm-abid.abderrazek-84557/ipykernel_1505554/1128040631.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load('best.pth')
/home/abid.abderrazek/.conda/e

In [17]:
all_preds=[]
all_labels=[]
for t_idx,t_val in enumerate(test_dataloader):
    val0=t_val
    out= model(input_ids=val0['input_ids'].to(device),attention_mask=val0['attention_mask'].to(device))
    targets=val0['labels']
    preds=torch.argmax(out.logits,dim=1)
    all_preds.append(preds)
    all_labels.append(targets)
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

NameError: name 'test_dataloader' is not defined

In [16]:
compute_f1(all_preds,all_labels)

{'f1_score': 0.8885496183206106}

In [18]:
tokenized_sub_dataset = dataset_test.remove_columns(['id', 'keyword', 'location', 'text'])


In [19]:

sub_dataloader=DataLoader(tokenized_sub_dataset['test'],batch_size=config['Eval_batch_size'],collate_fn=data_collator)
all_subs=[]
for t_idx,t_val in enumerate(sub_dataloader):
                val0=t_val
                out= model(input_ids=val0['input_ids'].to(device),attention_mask=val0['attention_mask'].to(device))
                preds=torch.argmax(out.logits,dim=1)
                all_subs.append(preds)
all_subs = torch.cat(all_subs)
all_subs,all_subs.shape


(tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0'), torch.Size([3263]))

In [21]:
import pandas as pd

submission_df = pd.DataFrame({
    'id': dataset_test['test']['id'],
    'target': list(all_subs.cpu().detach().numpy())
})
submission_df.to_csv('submission_2.csv', index=False)